> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [53]:
###imports 
import pandas as pd
from typing import List
from typing import Optional
from sqlalchemy import create_engine, URL
from sqlalchemy import text, select
from sqlalchemy import ForeignKey, String, Enum
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
from sqlalchemy.orm import sessionmaker
from enum import Enum
from datetime import date

In [46]:
###importando o banco
df = pd.read_csv('Salary_Prediction_Data_Professions.csv')
display(df)

,FIRST NAME,LAST NAME,SEX,DOJ,CURRENT DATE,DESIGNATION,AGE,SALARY,UNIT,LEAVES USED,LEAVES REMAINING,RATINGS,PAST EXP
0,TOMASA,ARMEN,F,5-18-2014,01-07-2016,Analyst,21.0,44570,Finance,24.0,6.0,2.0,0
1,ANNIE,NaN,F,NaN,01-07-2016,Associate,NaN,89207,Web,NaN,13.0,NaN,7
2,OLIVE,ANCY,F,7-28-2014,01-07-2016,Analyst,21.0,40955,Finance,23.0,7.0,3.0,0
3,CHERRY,AQUILAR,F,04-03-2013,01-07-2016,Analyst,22.0,45550,IT,22.0,8.0,3.0,0
4,LEON,ABOULAHOUD,M,11-20-2014,01-07-2016,Analyst,NaN,43161,Operations,27.0,3.0,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,KATHERINE,ALSDON,F,6-28-2011,01-07-2016,Senior Manager,36.0,185977,Management,15.0,15.0,5.0,10
2635,LOUISE,ALTARAS,F,1-14-2014,01-07-2016,Analyst,23.0,45758,IT,17.0,13.0,2.0,0
2636,RENEE,ALVINO,F,1-23-2014,01-07-2016,Analyst,21.0,47315,Web,29.0,1.0,5.0,0
2637,TERI,ANASTASIO,F,3-17-2014,01-07-2016,Analyst,24.0,45172,Web,23.0,7.0,3.0,1


#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [28]:
### buscando valores únicos de colunas específicas
lista_SEX = []

SEX_values = df["SEX"].drop_duplicates()
for value in SEX_values:
    lista_SEX.append(value)
    

lista_DESIGNATION = []

DESIGNATION_values = df["DESIGNATION"].drop_duplicates()
for value in DESIGNATION_values:
    lista_DESIGNATION.append(value)

lista_UNIT = []

UNIT_values = df["UNIT"].drop_duplicates()
for value in UNIT_values:
    lista_UNIT.append(value)
    
print(lista_SEX)
print(lista_DESIGNATION)
print(lista_UNIT)

['F', 'M']
['Analyst', 'Associate', 'Senior Analyst', 'Senior Manager', 'Manager', 'Director']
['Finance', 'Web', 'IT', 'Operations', 'Marketing', 'Management']


In [29]:
### buscando colunas com valores NaN no banco
colunas_com_nan = df.columns[df.isna().any()].tolist()

print("Colunas com valores NaN:", colunas_com_nan)
#assim, altera o campo nullable da classe para não ter erro ao importar os dados para a tabela

Colunas com valores NaN: ['LAST NAME', 'DOJ', 'AGE', 'LEAVES USED', 'LEAVES REMAINING', 'RATINGS']


In [30]:
### criando as classes Enum
class Sex(Enum):
    M = 'M'
    F = 'F'
    
class Designation(Enum):
    Analyst = 'Analyst'
    Associate = 'Associate'
    Senior_Analyst = 'Senior Analyst'
    Manager = 'Manager'
    Senior_Manager = 'Senior Manager'
    Director = 'Director'
    
class Unit(Enum):
    Finance = 'Finance'
    Web = 'Web'
    IT = 'IT'
    Operations = 'Operations'
    Marketing = 'Marketing'
    Management = 'Management'

In [35]:
### criando o ORM
class Base(DeclarativeBase):
    pass

class Funcionarios(Base):
    __tablename__ = 'employees'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str] = mapped_column(nullable=True)
    last_name: Mapped[str] = mapped_column(nullable=False)
    sex: Mapped[Sex] = mapped_column(nullable=False)
    doj: Mapped[date] = mapped_column(nullable=True) 
    current_date: Mapped[date] = mapped_column(nullable=False)
    designation: Mapped[Designation] = mapped_column(nullable=False)
    age: Mapped[int] = mapped_column(nullable=True)
    salary: Mapped[float] = mapped_column( nullable=False)
    unit: Mapped[Unit] = mapped_column(nullable=False)
    leaves_used: Mapped[int] = mapped_column(default=0, nullable=True)
    leaves_remaining: Mapped[int] = mapped_column(default=0, nullable=True)
    ratings: Mapped[float] = mapped_column(default=0.0, nullable=True)
    past_experience: Mapped[float] = mapped_column(default=0.0, nullable=False)


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [36]:
### entrando em contato
engine = create_engine('sqlite:///salarios.db', echo=True)

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [37]:
###  criando a tabela
Base.metadata.create_all(engine)

2025-01-16 15:47:20,911 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-16 15:47:20,913 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employees")
2025-01-16 15:47:20,914 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-16 15:47:20,916 INFO sqlalchemy.engine.Engine COMMIT


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [56]:
### populando o banco salarios
df.to_sql('funcionarios', engine, if_exists='append', index=False)

2025-01-16 16:04:17,178 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-16 16:04:17,188 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("funcionarios")
2025-01-16 16:04:17,189 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-16 16:04:17,211 INFO sqlalchemy.engine.Engine INSERT INTO funcionarios ("FIRST NAME", "LAST NAME", "SEX", "DOJ", "CURRENT DATE", "DESIGNATION", "AGE", "SALARY", "UNIT", "LEAVES USED", "LEAVES REMAINING", "RATINGS", "PAST EXP") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-01-16 16:04:17,212 INFO sqlalchemy.engine.Engine [generated in 0.01223s] [('TOMASA', 'ARMEN', 'F', '5-18-2014', '01-07-2016', 'Analyst', 21.0, 44570, 'Finance', 24.0, 6.0, 2.0, 0), ('ANNIE', None, 'F', None, '01-07-2016', 'Associate', None, 89207, 'Web', None, 13.0, None, 7), ('OLIVE', 'ANCY', 'F', '7-28-2014', '01-07-2016', 'Analyst', 21.0, 40955, 'Finance', 23.0, 7.0, 3.0, 0), ('CHERRY', 'AQUILAR', 'F', '04-03-2013', '01-07-2016', 'Analyst', 22.0, 45550, 'IT', 22.0, 8.0, 3.0,

2639

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [67]:
### query SQL com engine.connect
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM funcionarios")).fetchall()
                                                
    for row in result:
        print(row)

2025-01-16 16:11:00,961 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-16 16:11:00,963 INFO sqlalchemy.engine.Engine SELECT * FROM funcionarios
2025-01-16 16:11:00,966 INFO sqlalchemy.engine.Engine [cached since 1164s ago] ()
('TOMASA', 'ARMEN', 'F', '5-18-2014', '01-07-2016', 'Analyst', 21.0, 44570, 'Finance', 24.0, 6.0, 2.0, 0)
('ANNIE', None, 'F', None, '01-07-2016', 'Associate', None, 89207, 'Web', None, 13.0, None, 7)
('OLIVE', 'ANCY', 'F', '7-28-2014', '01-07-2016', 'Analyst', 21.0, 40955, 'Finance', 23.0, 7.0, 3.0, 0)
('CHERRY', 'AQUILAR', 'F', '04-03-2013', '01-07-2016', 'Analyst', 22.0, 45550, 'IT', 22.0, 8.0, 3.0, 0)
('LEON', 'ABOULAHOUD', 'M', '11-20-2014', '01-07-2016', 'Analyst', None, 43161, 'Operations', 27.0, 3.0, None, 3)
('VICTORIA', None, 'F', '2-19-2013', '01-07-2016', 'Analyst', 22.0, 48736, 'Marketing', 20.0, 10.0, 4.0, 0)
('ELLIOT', 'AGULAR', 'M', '09-02-2013', '01-07-2016', 'Analyst', 22.0, 40339, 'Marketing', 19.0, 11.0, 5.0, 0)
('JACQUES', 'AKMAL', 'M'

In [68]:
### query SQL com Pandas
with engine.connect() as conn:
    query = "SELECT * FROM funcionarios;"
    
    df = pd.read_sql_query(query, con=conn)

display(df)

2025-01-16 16:11:03,008 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-16 16:11:03,010 INFO sqlalchemy.engine.Engine SELECT * FROM funcionarios;
2025-01-16 16:11:03,011 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-16 16:11:03,040 INFO sqlalchemy.engine.Engine ROLLBACK


,FIRST NAME,LAST NAME,SEX,DOJ,CURRENT DATE,DESIGNATION,AGE,SALARY,UNIT,LEAVES USED,LEAVES REMAINING,RATINGS,PAST EXP
0,TOMASA,ARMEN,F,5-18-2014,01-07-2016,Analyst,21.0,44570,Finance,24.0,6.0,2.0,0
1,ANNIE,None,F,None,01-07-2016,Associate,NaN,89207,Web,NaN,13.0,NaN,7
2,OLIVE,ANCY,F,7-28-2014,01-07-2016,Analyst,21.0,40955,Finance,23.0,7.0,3.0,0
3,CHERRY,AQUILAR,F,04-03-2013,01-07-2016,Analyst,22.0,45550,IT,22.0,8.0,3.0,0
4,LEON,ABOULAHOUD,M,11-20-2014,01-07-2016,Analyst,NaN,43161,Operations,27.0,3.0,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5273,KATHERINE,ALSDON,F,6-28-2011,01-07-2016,Senior Manager,36.0,185977,Management,15.0,15.0,5.0,10
5274,LOUISE,ALTARAS,F,1-14-2014,01-07-2016,Analyst,23.0,45758,IT,17.0,13.0,2.0,0
5275,RENEE,ALVINO,F,1-23-2014,01-07-2016,Analyst,21.0,47315,Web,29.0,1.0,5.0,0
5276,TERI,ANASTASIO,F,3-17-2014,01-07-2016,Analyst,24.0,45172,Web,23.0,7.0,3.0,1


In [79]:
### query SQL com SQLAlchemy ORM

# Criar uma sessão
Session = sessionmaker(bind=engine)
session = Session()

query = select(Funcionario)
result = session.execute(query).all() # o mesmo que SELECT * FROM funcionarios;

for row in result: 
    print(row.Funcionario.id, row.Funcionario.first_name, row.Funcionario.age, row.Funcionario.designation)

session.commit()

# Fechar a sessão 
session.close()

2025-01-16 16:15:03,197 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-16 16:15:03,200 INFO sqlalchemy.engine.Engine SELECT employees.id, employees.first_name, employees.last_name, employees.sex, employees.doj, employees."current_date", employees.designation, employees.age, employees.salary, employees.unit, employees.leaves_used, employees.leaves_remaining, employees.ratings, employees.past_experience 
FROM employees
2025-01-16 16:15:03,201 INFO sqlalchemy.engine.Engine [cached since 665.2s ago] ()
2025-01-16 16:15:03,203 INFO sqlalchemy.engine.Engine COMMIT
